# SQL Alchemy
SQL Alchemy is an SQL toolkit, an object relational mapper, a python module, that allows us to map python classes and objects to database tables and entries so we don't have to write any SQL code 

first we must download sql alchemy by opening up terminal and running the command `pip install sqlalchemy`

In [120]:
from sqlalchemy import create_engine, ForeignKey, Column, String, Integer, CHAR
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker

In [121]:
# this is going to return a class
Base = declarative_base()

/var/folders/5z/f5jycvmd0x58dvn1by8xk79h0000gn/T/ipykernel_29438/864283243.py:2: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [122]:
# Person class will inherit from Base
# usually specify plural when naming tables
class Person(Base):
    __tablename__ = "people"
    ssn = Column("ssn", Integer, primary_key=True)
    firstname = Column("firstname", String)
    lastname = Column("lastname", String)
    gender = Column("gender", CHAR)
    age = Column("age", Integer)

    # constructor
    def __init__(self, ssn, first, last, gender, age):
        self.ssn = ssn
        self.firstname = first
        self.lastname = last
        self.gender = gender
        self.age = age

    # allows us to specify how we want to print a person
    def __repr__(self):
        return f"({self.ssn}) {self.firstname} {self.lastname} ({self.gender},{self.age})"

In [123]:
class Thing(Base):
    __tablename__ = "things"

    tid = Column("tid", Integer, primary_key=True)
    description = Column("description", String)
    # owner refers to ssn of people class
    owner = Column(Integer, ForeignKey("people.ssn"))

    def __init__(self, tid, description, owner):
        self.tid = tid
        self.description = description
        self.owner = owner

    def __repr__(self):
        return f"({self.tid}) {self.description} owned by {self.owner}"
    

In [124]:
# The create_engine function is used to establish a connection to the database.
# echo=True: This argument enables SQLAlchemy's logging feature, which will print all the generated SQL queries to the console. This can be useful for debugging purposes.
engine = create_engine("sqlite:///mydb.db", echo=True)
# below code creates the table
Base.metadata.create_all(bind=engine)

# class
Session = sessionmaker(bind=engine)
# instance
session = Session()

2024-08-20 09:29:27,800 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-08-20 09:29:27,802 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("people")
2024-08-20 09:29:27,802 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-08-20 09:29:27,806 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("people")
2024-08-20 09:29:27,807 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-08-20 09:29:27,808 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("things")
2024-08-20 09:29:27,808 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-08-20 09:29:27,809 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("things")
2024-08-20 09:29:27,809 INFO sqlalchemy.engine.Engine [raw sql] ()
2024-08-20 09:29:27,811 INFO sqlalchemy.engine.Engine 
CREATE TABLE people (
	ssn INTEGER NOT NULL, 
	firstname VARCHAR, 
	lastname VARCHAR, 
	gender CHAR, 
	age INTEGER, 
	PRIMARY KEY (ssn)
)


2024-08-20 09:29:27,811 INFO sqlalchemy.engine.Engine [no key 0.00039s] ()
2024-08-20 09:29:27,814 INFO sqlalchemy.eng

In [125]:
# procedure for creating a person (need to add and commit)
person = Person(12312, "Timothy", "Park", "m", 32)
session.add(person)
session.commit()

p1 = Person(31234, "James", "Harrison", "m", 46)
p2 = Person(32423, "Jimmy", "Butler", "m", 34)
p3 = Person(45654, "Larry", "Wheels", "m", 29)

session.add(p1)
session.add(p2)
session.add(p3)
session.commit()

2024-08-20 09:29:27,826 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-08-20 09:29:27,829 INFO sqlalchemy.engine.Engine INSERT INTO people (ssn, firstname, lastname, gender, age) VALUES (?, ?, ?, ?, ?)
2024-08-20 09:29:27,829 INFO sqlalchemy.engine.Engine [generated in 0.00099s] (12312, 'Timothy', 'Park', 'm', 32)
2024-08-20 09:29:27,832 INFO sqlalchemy.engine.Engine COMMIT
2024-08-20 09:29:27,840 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-08-20 09:29:27,841 INFO sqlalchemy.engine.Engine INSERT INTO people (ssn, firstname, lastname, gender, age) VALUES (?, ?, ?, ?, ?)
2024-08-20 09:29:27,841 INFO sqlalchemy.engine.Engine [generated in 0.00046s] [(31234, 'James', 'Harrison', 'm', 46), (32423, 'Jimmy', 'Butler', 'm', 34), (45654, 'Larry', 'Wheels', 'm', 29)]
2024-08-20 09:29:27,843 INFO sqlalchemy.engine.Engine COMMIT


In [126]:
# we can execute queries
results = session.query(Person).all()
print(results)

2024-08-20 09:29:27,849 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-08-20 09:29:27,852 INFO sqlalchemy.engine.Engine SELECT people.ssn AS people_ssn, people.firstname AS people_firstname, people.lastname AS people_lastname, people.gender AS people_gender, people.age AS people_age 
FROM people
2024-08-20 09:29:27,853 INFO sqlalchemy.engine.Engine [generated in 0.00088s] ()
[(12312) Timothy Park (m,32), (31234) James Harrison (m,46), (32423) Jimmy Butler (m,34), (45654) Larry Wheels (m,29)]


In [127]:
# filter by string
results = session.query(Person).filter(Person.lastname == "Park")
for r in results:
    print(r)

2024-08-20 09:29:27,861 INFO sqlalchemy.engine.Engine SELECT people.ssn AS people_ssn, people.firstname AS people_firstname, people.lastname AS people_lastname, people.gender AS people_gender, people.age AS people_age 
FROM people 
WHERE people.lastname = ?
2024-08-20 09:29:27,862 INFO sqlalchemy.engine.Engine [generated in 0.00136s] ('Park',)
(12312) Timothy Park (m,32)


In [128]:
# filter by integer
results = session.query(Person).filter(Person.age > 32)
for r in results:
    print(r)

2024-08-20 09:29:27,871 INFO sqlalchemy.engine.Engine SELECT people.ssn AS people_ssn, people.firstname AS people_firstname, people.lastname AS people_lastname, people.gender AS people_gender, people.age AS people_age 
FROM people 
WHERE people.age > ?
2024-08-20 09:29:27,872 INFO sqlalchemy.engine.Engine [generated in 0.00104s] (32,)
(31234) James Harrison (m,46)
(32423) Jimmy Butler (m,34)


In [129]:
# filter by "like"
results = session.query(Person).filter(Person.firstname.like("%Jim%"))
for r in results:
    print(r)

2024-08-20 09:29:27,879 INFO sqlalchemy.engine.Engine SELECT people.ssn AS people_ssn, people.firstname AS people_firstname, people.lastname AS people_lastname, people.gender AS people_gender, people.age AS people_age 
FROM people 
WHERE people.firstname LIKE ?
2024-08-20 09:29:27,880 INFO sqlalchemy.engine.Engine [generated in 0.00093s] ('%Jim%',)
(32423) Jimmy Butler (m,34)


In [130]:
# filter by list
results = session.query(Person).filter(Person.firstname.in_(["Timothy", "James"]))
for r in results:
    print(r)

2024-08-20 09:29:27,886 INFO sqlalchemy.engine.Engine SELECT people.ssn AS people_ssn, people.firstname AS people_firstname, people.lastname AS people_lastname, people.gender AS people_gender, people.age AS people_age 
FROM people 
WHERE people.firstname IN (?, ?)
2024-08-20 09:29:27,887 INFO sqlalchemy.engine.Engine [generated in 0.00160s] ('Timothy', 'James')
(12312) Timothy Park (m,32)
(31234) James Harrison (m,46)


In [131]:
t1 = Thing(1, "Car", p1.ssn)
t2 = Thing(2, "Laptop", p1.ssn)
t3 = Thing(3, "PS5", p2.ssn)
t4 = Thing(4, "Tool", p3.ssn)
t5 = Thing(5, "Book", p3.ssn)

session.add(t1)
session.add(t2)
session.add(t3)
session.add(t4)
session.add(t5)
session.commit()

2024-08-20 09:29:27,897 INFO sqlalchemy.engine.Engine INSERT INTO things (tid, description, owner) VALUES (?, ?, ?)
2024-08-20 09:29:27,898 INFO sqlalchemy.engine.Engine [generated in 0.00080s] [(1, 'Car', 31234), (2, 'Laptop', 31234), (3, 'PS5', 32423), (4, 'Tool', 45654), (5, 'Book', 45654)]
2024-08-20 09:29:27,899 INFO sqlalchemy.engine.Engine COMMIT


In [132]:
results = session.query(Thing, Person).filter(Thing.owner == Person.ssn).filter(Person.firstname == "James").all()
for r in results:
    print(r)

2024-08-20 09:29:27,906 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2024-08-20 09:29:27,908 INFO sqlalchemy.engine.Engine SELECT things.tid AS things_tid, things.description AS things_description, things.owner AS things_owner, people.ssn AS people_ssn, people.firstname AS people_firstname, people.lastname AS people_lastname, people.gender AS people_gender, people.age AS people_age 
FROM things, people 
WHERE things.owner = people.ssn AND people.firstname = ?
2024-08-20 09:29:27,909 INFO sqlalchemy.engine.Engine [generated in 0.00082s] ('James',)
((1) Car owned by 31234, (31234) James Harrison (m,46))
((2) Laptop owned by 31234, (31234) James Harrison (m,46))
